In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_parquet('data/profiles/stackoverflow')

In [3]:
df.head()

,user_name,headline,work,details,tags,date_added
0,Gabriel Lincoln,g97santos@gmail.com\nhttps://github.com/LincolnG4,Automotive data engineer at Bosch,"{' GitHub ': 'https://github.com/LincolnG4', ...","[can-bus, python, python-can, python-3.x, obd-ii]",2024-03-10
1,GabrielBoehme,Software Developer and Data Scientist/Engineer...,Data Engineer and Full Stack Engineer,"{' GitHub ': None, ' Twitter ': None, 'candy...","[pandas, string, airflow, amazon-web-services,...",2024-03-10
2,Tung Nguyen,Email: tung.nn83@gmail.com\nMobile: 84 907 193...,Data Engineer at Vincere.io,"{' GitHub ': 'https://github.com/itnnt', ' T...","[python, pyspark, apache-spark, r, multiproces...",2024-03-10
3,Purushothaman Srikanth,Let's discuss Data Engineering.\nI come to SO ...,Data Engineer @ Salesforce,"{' GitHub ': None, ' Twitter ': None, 'candy...","[deep-learning, pytorch, pickle, serialization...",2024-03-10
4,Jose Manuel de Frutos,Graduated in Mathematics and Computer Science....,Full Stack Software Engineer at Devo inc,{' GitHub ': 'https://github.com/josemanuel22...,"[pandas, julia, math, javascript, machine-lear...",2024-03-10


In [4]:
for k, v in df['details'][0].items():
    print(k, ":",v)



GitHub
 : https://github.com/LincolnG4


Twitter
 : None
candycrush.wikia.com/wiki/… : None
careers.stackoverflow.com/… : None
cmaliwal.weebly.com : None
donkirkby.github.io : None
github.com/amirsoroush : None
in.linkedin.com/in/… : None
josemanuel22.github.io : None
liberapay.com/isalgueiro/… : None
linkedin.com/in/sarwarhayat : None
linkedin.com/in/… : https://www.linkedin.com/in/gabriel-espinola-lincoln/?locale=en_US
navaneethnanda.com : None
ncapito.ghost.io : None
snippetnuggets.com : None
upwork.com/freelancers/… : None
w3farid.github.io/portfolio : None


In [5]:
def preprocess_details(details):
    urls = []
    new_details = {}
    for k, v in details.items():
        new_details[k.strip()] = v

    features = ['linkedin.com/in/…', 'GitHub']
    for f in features:
        if f in new_details:
            urls.append(new_details[f])

    urls = [i for i in urls if i != None]
    return {'websites': urls}

In [6]:
def extract_email(text):
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    email = re.search(email_pattern, text)
    return email.group() if email else None

In [7]:
def preprocess(x):
    data = {}
    data['user_name'] = x['user_name']
    data['headline'] = x['headline']
    data['work'] = x['work']
    data['email'] = extract_email(x['headline'])
    data.update(preprocess_details(x['details']))
    data['skills'] = x['tags']
    data['date_added'] = x['date_added']
    return data

In [8]:
preprocessed_data = df.apply(lambda x: preprocess(x), axis=1).to_list()

In [9]:
preprocessed_df = pd.DataFrame(preprocessed_data)

In [10]:
preprocessed_df.head()

,user_name,headline,work,email,websites,skills,date_added
0,Gabriel Lincoln,g97santos@gmail.com\nhttps://github.com/LincolnG4,Automotive data engineer at Bosch,g97santos@gmail.com,[https://www.linkedin.com/in/gabriel-espinola-...,"[can-bus, python, python-can, python-3.x, obd-ii]",2024-03-10
1,GabrielBoehme,Software Developer and Data Scientist/Engineer...,Data Engineer and Full Stack Engineer,gabboehme@gmail.com,[],"[pandas, string, airflow, amazon-web-services,...",2024-03-10
2,Tung Nguyen,Email: tung.nn83@gmail.com\nMobile: 84 907 193...,Data Engineer at Vincere.io,tung.nn83@gmail.com,[https://github.com/itnnt],"[python, pyspark, apache-spark, r, multiproces...",2024-03-10
3,Purushothaman Srikanth,Let's discuss Data Engineering.\nI come to SO ...,Data Engineer @ Salesforce,jispurushothaman@gmail.com,[https://linkedin.com/in/purushothaman-srikanth],"[deep-learning, pytorch, pickle, serialization...",2024-03-10
4,Jose Manuel de Frutos,Graduated in Mathematics and Computer Science....,Full Stack Software Engineer at Devo inc,josemanuel.defrutos22@gmail.com,[https://github.com/josemanuel22],"[pandas, julia, math, javascript, machine-lear...",2024-03-10


In [11]:
preprocessed_df.to_parquet('data/preprocessed_profiles/stackoverflow', partition_cols=['date_added'])